# Point Cloud Visibility Analysis

## Setup

In [1]:
import glob
import laspy
import numpy
import open3d
from scipy.spatial import KDTree
import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Configuration
las_folder = glob.glob("LAS/*.las")
downsample_size = 0   # Set to 0 for no downsampling
observer_xyz = [120, 0, 5]
sampling_divisions = 100
search_radius = 0.2
save_original_ply = True
save_result_ply = True

In [3]:
# Merge LAS files into one point cloud
points = []
for f in las_folder:
    las = laspy.read(f)
    x = las.X * las.header.scales[0] + las.header.offsets[0]
    y = las.Y * las.header.scales[1] + las.header.offsets[1]
    z = las.Z * las.header.scales[2] + las.header.offsets[2]
    points.append(numpy.vstack([x, y, z]).T)
points = numpy.vstack(points)
print(f"{len(points)} points merged.")
pointcloud = open3d.geometry.PointCloud()
pointcloud.points = open3d.utility.Vector3dVector(numpy.vstack(points))

13448516 points merged.


In [4]:
# Optional polygon file and downsampling
if save_original_ply:
    open3d.io.write_point_cloud("pointcloud.ply", pointcloud)
if downsample_size > 0:
    pointcloud = pointcloud.voxel_down_sample(downsample_size)
    print(f"Downsampled to {len(pointcloud.points)} points.")
points = numpy.asarray(pointcloud.points)

## Visibility analysis

In [5]:
# Set observer point and search tree
box_center = pointcloud.get_axis_aligned_bounding_box().get_center()
observer = numpy.array([box_center[0] + observer_xyz[0], box_center[1] + observer_xyz[1], box_center[2] + observer_xyz[2]])
kdtree = open3d.geometry.KDTreeFlann(pointcloud)
visible_points = []
for p in tqdm.tqdm(points):  # tqdm is a progress bar
    # Calculate and divide vector from observer to point
    distance = numpy.linalg.norm(p - observer)
    vector = (p - observer) / distance
    divisions = numpy.linspace(0, distance, sampling_divisions)[1:-1]  # Ray is divided
    blocked = False
    # Check along each division for obstructing points
    for d in (divisions[:, None] * vector + observer):
        srv, _, _ = kdtree.search_radius_vector_3d(d, 0.4)
        if srv > 0:
            blocked = True
            break
    # Not obstructed = visible
    visible_points.append(not blocked)
visible_points = numpy.array(visible_points)

100%|██████████████████████████████████████████████████████████████████| 13448516/13448516 [2:23:11<00:00, 1565.42it/s]


In [6]:
print(f"{numpy.sum(visible_points) / len(points) * 100:.2f}% visible from observer point.")

5.12% visible from observer point.


## Display result

In [7]:
# Color point cloud
colors = numpy.zeros_like(points)
colors[visible_points] = [0, 1, 0]  # green
colors[~visible_points] = [1, 0, 0]  # red
pointcloud.colors = open3d.utility.Vector3dVector(colors)

In [8]:
# Draw observer point
obs_point = open3d.geometry.TriangleMesh.create_sphere(3)
obs_point.paint_uniform_color([0, 0, 1])  # blue
obs_point.translate(observer);

In [9]:
open3d.visualization.draw_geometries([pointcloud, obs_point])

## Save result

In [10]:
# Optional polygon file
if save_result_ply:
    open3d.io.write_point_cloud("visibility.ply", pointcloud)

In [11]:
# Write to LAS
header = laspy.LasHeader()
header.scales = numpy.array([0.001, 0.001, 0.001])
header.offsets = numpy.min(points, axis=0)
las = laspy.LasData(header)
las.x, las.y, las.z = points[:, 0], points[:, 1], points[:, 2]
scaled_colors = (colors * 65535).astype(numpy.uint16)
las.red, las.green, las.blue = scaled_colors[:, 0], scaled_colors[:, 1], scaled_colors[:, 2]
las.write("visibility.las")